In [9]:
# Тюнинг
import optuna as opt
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.cluster import KMeans

# Пайплайн
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator

# Данные
import os
import pandas as pd
import numpy as np
from category_encoders import BinaryEncoder
from sklearn.preprocessing import RobustScaler

In [31]:
# Пути
ROOT = os.getcwd()
TRAIN_DATASET = os.path.join(ROOT, '../data/train_AIC.csv')
BALANCED_DATASET = os.path.join(ROOT, '../data/balanced_train.csv')
TEST_DATASET = os.path.join(ROOT, '../data/test_AIC.csv')

# Загрузка
train_df = pd.read_csv(TRAIN_DATASET)
balanced_df = pd.read_csv(BALANCED_DATASET, index_col=0)
test_df = pd.read_csv(TEST_DATASET)

X, y = balanced_df.iloc[:, :-1], balanced_df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
# X_train = X_train.drop([
#     'Категорийный менеджер', 'Изменение позиции заказа на закупку: изменение даты поставки на бумаге',
#     'Количество', 'Дней между 0_1', 'Дней между 1_2', 'Дней между 2_3', 'Дней между 3_4', 'Дней между 4_5', 
#     'Дней между 5_6', 'Дней между 6_7', 'Дней между 7_8', 'Согласование заказа 1', 'Согласование заказа 2',
#     'Согласование заказа 3', 'Изменение даты поставки 7', 'Изменение даты поставки 15', 'Изменение даты поставки 30'
#     ], axis=1)
# X_test = X_test.drop([
#     'Категорийный менеджер', 'Изменение позиции заказа на закупку: изменение даты поставки на бумаге',
#     'Количество', 'Дней между 0_1', 'Дней между 1_2', 'Дней между 2_3', 'Дней между 3_4', 'Дней между 4_5', 
#     'Дней между 5_6', 'Дней между 6_7', 'Дней между 7_8', 'Согласование заказа 1', 'Согласование заказа 2',
#     'Согласование заказа 3', 'Изменение даты поставки 7', 'Изменение даты поставки 15', 'Изменение даты поставки 30'
#     ], axis=1)

In [32]:
cat_features = [
    'Поставщик', 'Материал', 'Операционный менеджер',
    'Завод', 'Закупочная организация', 'Группа закупок', 'Балансовая единица',
    'ЕИ', 'Вариант поставки'
    ]

In [33]:
# Препроцессоры
class DataPreprocessor(BaseEstimator, TransformerMixin):
    """ Предобработчик данных """
    def __init__(self, cat_features, transform_train=True):
        self.transform_train = transform_train
        self.bin_encoder = BinaryEncoder(cols=cat_features)
        self.robust_scaler = RobustScaler()

    def fit(self, X, y=None):
        # Создаём копию датасета
        X_ = X.copy()

        # Временные фичи
        X_['day_sin'] = np.sin(np.pi * 2 * X_['День недели 2'] / 6)
        X_['day_cos'] = np.cos(np.pi * 2 * X_['День недели 2'] / 6)

        # Категориальные фичи
        X_ = self.bin_encoder.fit_transform(X_)

        # Нумерация фич для LGBM (не принимает JSON символы)
        X_.columns = [num for num in range(0, len(X_.columns))]

        # Масштабирование
        self.robust_scaler.fit(X_)

        return self
    
    def transform(self, X):
        # Создаём копию датасета
        X_ = X.copy()

        # Временные фичи
        X_['day_sin'] = np.sin(np.pi * 2 * X_['День недели 2'] / 6)
        X_['day_cos'] = np.cos(np.pi * 2 * X_['День недели 2'] / 6)

        # Категориальные фичи
        X_ = self.bin_encoder.transform(X_)

        # Нумерация фич для LGBM (не принимает JSON символы)
        X_.columns = [num for num in range(0, len(X_.columns))]

        # Масштабирование
        X_ = self.robust_scaler.transform(X_)

        return X_


In [19]:
# data_preprocessor.fit(X_train)
# X_train_preproc = data_preprocessor.transform(X_train)
# X_test_preproc = data_preprocessor.transform(X_test)

In [47]:
# Функция оптимизации
def objective(trial: opt.Trial):
    # Параметры
    learning_rate = trial.suggest_float('learning_rate', 0.1, 1, log=True)
    n_estimators = trial.suggest_int('n_estimators', 1000, 3000, 50)
    max_depth = trial.suggest_int('max_depth', 6, 24)
    max_bin = trial.suggest_int('max_bin', 64, 256),
    num_leaves = trial.suggest_int('num_leaves', 64, 512)
    reg_lambda = trial.suggest_float('l2_reg', 0.1, 1.5, log=True)

    # Модель
    data_preprocessor = DataPreprocessor(cat_features)
    model = LGBMClassifier(
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        max_depth=max_depth,
        reg_lambda=reg_lambda,
        max_bin=max_bin,
        n_jobs=-1,
        force_col_wise=True,
        verbose=-1
    )

    pipeline = Pipeline([
        ('data_preproc', data_preprocessor),
        ('model', model)
    ])

    pipeline.fit(X_train, y_train)
    accuracy = f1_score(y_test, pipeline.predict(X_test), average='macro')
    
    # cv_score = cross_val_score(model, X_train_lgbm, y_train, cv=StratifiedKFold(), scoring='f1_macro', n_jobs=-1)
    # accuracy = cv_score.mean()

    return accuracy

In [ ]:
X_train_lgbm

In [ ]:
study = opt.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

In [ ]:
study.best_params

In [34]:
X_train.index.value_counts()

203033     2
45         2
192578     2
132212     2
88582      2
          ..
193789     1
298564     1
158332     1
2176351    1
121958     1
Name: count, Length: 321762, dtype: int64

In [45]:
best_params = {
    'learning_rate': 0.3, 
    'n_estimators': 1100, 
    'max_depth': 12, 
    'max_bin': 204, 
    'num_leaves': 304, 
    'reg_lambda': 0.6
    }

# Модель
data_preprocessor = DataPreprocessor(cat_features)
model = LGBMClassifier(
    **best_params,
    n_jobs=-1,
    force_col_wise=True,
)

pipeline = Pipeline([
    ('data_preproc', data_preprocessor),
    ('model', model)
])

In [46]:
pipeline.fit(X_train, y_train)
f1_score(y_test, pipeline.predict(X_test))

[LightGBM] [Info] Number of positive: 174731, number of negative: 173285
[LightGBM] [Info] Total Bins 2837
[LightGBM] [Info] Number of data points in the train set: 348016, number of used features: 102
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502077 -> initscore=0.008310
[LightGBM] [Info] Start training from score 0.008310
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

0.9889264841587203

In [47]:
f1_score(y_test, pipeline.predict(X_test), average='macro')

0.9888273813059948

In [48]:
X_train.shape

(348016, 43)

In [19]:
test_df

,Поставщик,Материал,Операционный менеджер,Завод,Закупочная организация,Группа закупок,Балансовая единица,ЕИ,Группа материалов,Вариант поставки,...,День недели 2,Сумма,Количество позиций,Количество обработчиков 7,Количество обработчиков 15,Количество обработчиков 30,Отмена полного деблокирования заказа на закупку,Изменение позиции заказа на закупку: дата поставки,Количество циклов согласования,Количество изменений после согласований
0,273,269,8,1,1,64,1,9,22,1,...,6,6.560264,2,8,8,8,0,0,1.0,16
1,499,27439,10,18,16,1,14,1,35,2,...,3,4.362439,6,4,4,4,0,0,1.0,12
2,86,27439,2,1,1,172,1,1,26,2,...,4,7.315035,1,6,6,6,0,1,1.0,12
3,97,4064,4,7,1,22,6,3,46,1,...,3,2.883392,6,7,8,8,0,1,1.0,16
4,117,27439,3,1,1,2,1,1,2,1,...,3,7.333641,7,7,7,7,0,1,1.0,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,255,6409,6,3,1,10,3,1,16,2,...,6,6.537037,5,10,10,10,1,0,2.0,18
24996,1,5827,1,7,1,46,6,1,13,1,...,3,7.425021,4,8,8,8,1,1,2.0,0
24997,8,6504,5,3,1,17,3,1,21,2,...,3,5.703462,13,6,6,6,0,0,1.0,9
24998,18,1309,17,1,1,44,1,1,142,2,...,3,9.059027,6,9,10,10,2,0,3.0,31


In [24]:
test_df_sub

,Поставщик,Материал,Операционный менеджер,Завод,Закупочная организация,Группа закупок,Балансовая единица,ЕИ,Группа материалов,Вариант поставки,...,День недели 2,Сумма,Количество позиций,Количество обработчиков 7,Количество обработчиков 15,Количество обработчиков 30,Отмена полного деблокирования заказа на закупку,Изменение позиции заказа на закупку: дата поставки,Количество циклов согласования,Количество изменений после согласований
0,273,269,8,1,1,64,1,9,22,1,...,6,6.560264,2,8,8,8,0,0,1.0,16
1,499,27439,10,18,16,1,14,1,35,2,...,3,4.362439,6,4,4,4,0,0,1.0,12
2,86,27439,2,1,1,172,1,1,26,2,...,4,7.315035,1,6,6,6,0,1,1.0,12
3,97,4064,4,7,1,22,6,3,46,1,...,3,2.883392,6,7,8,8,0,1,1.0,16
4,117,27439,3,1,1,2,1,1,2,1,...,3,7.333641,7,7,7,7,0,1,1.0,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,255,6409,6,3,1,10,3,1,16,2,...,6,6.537037,5,10,10,10,1,0,2.0,18
24996,1,5827,1,7,1,46,6,1,13,1,...,3,7.425021,4,8,8,8,1,1,2.0,0
24997,8,6504,5,3,1,17,3,1,21,2,...,3,5.703462,13,6,6,6,0,0,1.0,9
24998,18,1309,17,1,1,44,1,1,142,2,...,3,9.059027,6,9,10,10,2,0,3.0,31


In [28]:
test_df.columns.isin(train_df.columns)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [30]:
train_df['y']

0         1
1         0
2         0
3         0
4         0
         ..
224995    0
224996    0
224997    0
224998    0
224999    0
Name: y, Length: 225000, dtype: int64

In [50]:
test_df_sub = test_df
preds = pipeline.predict(test_df_sub)

In [51]:
submit_df = pd.DataFrame({'id': test_df.index, 'value': preds})

In [52]:
submit_df.to_csv('submission_3.csv', index=False)

In [29]:
submit_df

,id,value
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
24995,24995,0
24996,24996,1
24997,24997,1
24998,24998,1
